In [ ]:
"{}sadas".format("What is")

'What issadas'

In [1]:
def preprocess_data(data, input_template=None, input_key="input", apply_chat_template=None) -> str:
    if apply_chat_template:
        prompt = apply_chat_template(data[input_key], tokenize=False, add_generation_prompt=True)
    else:
        prompt = data[input_key]
        if input_template:
            prompt = input_template.format(prompt)
    return prompt

In [1]:
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from RewardModel import SentencePairDataset, RoBERTaRewardModel
from argparse import Namespace

from openrlhf.utils import get_strategy

args = Namespace(
    zero_stage=3,
    save_path="./models/"
)

strategy = get_strategy(args)


tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

data = [
    ("The sun is shining", "It is a bright day", "It is raining"),
    ("The sun is shining", "It is a bright day", "It is raining"),
]
dataset = SentencePairDataset(data, tokenizer, max_length=16)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# model = RoBERTaRewardModel()

model = RobertaModel.from_pretrained("roberta-large")

optimizer = Adam(params =  model.parameters(), lr=0.1)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

/home/adrian/Projects/RLHF4Hate/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-06 11:42:21,194] [WARNING] [real_accelerator.py:162:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2024-12-06 11:42:21,197] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cpu (auto detect)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
import torch.distributed as dist


def init_distributed_mode():
    if torch.cuda.is_available():
        dist.init_process_group(backend='nccl')  # 'nccl' is typically used for GPU
    else:
        dist.init_process_group(backend='gloo')  # 'gloo' is used for CPU
init_distributed_mode()

ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set

In [4]:
from openrlhf.trainer import RewardModelTrainer

trainer = RewardModelTrainer(
    model=model,
    strategy=strategy, # WTF is this
    optim=optimizer,
    train_dataloader=dataloader,
    eval_dataloader=dataloader, # TODO: eval split
    scheduler=scheduler,
    tokenizer=tokenizer,
    max_norm=0.5,
    max_epochs=2,
    loss="sigmoid",
)

2024-12-06 11:42:41,551	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ValueError: Default process group has not been initialized, please make sure to call init_process_group.